We use a product review dataset

In [ ]:
!wget https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/PROD.csv

--2022-01-07 19:01:50--  https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/PROD.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 527639 (515K) [text/plain]
Saving to: ‘PROD.csv’

PROD.csv            100%[===================>] 515.27K  --.-KB/s    in 0.04s   

2022-01-07 19:01:50 (13.5 MB/s) - ‘PROD.csv’ saved [527639/527639]



## Imports

In [ ]:
import tensorflow as tf
import re
import numpy as np
import csv 
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle


## Read the Dataset

preprocess a review by removing special characters and long spaces

In [ ]:
def process_review(review):
  out = re.sub(r"[^\w\s]", '', review)
  out = re.sub(r"[a-zA-Z]", '', out)
  out = re.sub(r"\n", '', out)
  out = re.sub(r"\s+", ' ', out)
  return out.strip()

In [ ]:
with open('PROD.csv', 'r') as csv_file:
  reviews = []
  labels  = []
  all_text = ""
  count = 0
  pos_count = 0
  
  #read the data
  lines = csv.reader(csv_file, delimiter = ",")
  for i, line in enumerate(lines):
    
    #ignore the first line
    if i == 0: continue
    
    #preprocess the data
    review = process_review(line[0])
    label  = int(line[1])
    
    #only allow postiive and negative reviews, 
    #also make them the same length
    
    if label == 1:
      pos_count +=1
    elif label == 0:
      continue
    else:
      label += 1
      
    if label == 1 and pos_count > 862:
      continue
    
    if review == "":
      continue
    reviews.append(review)
    all_text += review +' \n '
    labels.append(label)
    
#shuffle the data
reviews, labels = shuffle(reviews, labels)
print(len(reviews))

1648


Look at the data

In [ ]:
for i in range(0, 10):
  print(reviews[i], labels[i])

ممتازة ورائعة 1
السلعة جيدة ولكن ليست السعلة التى اختارتها مو اول مرة تسوها معى 0
رأيت هذه السلعة في أسواق التجهيزات الصوتية بسعر لا يتعدى ال 1150 ريال سعودي 0
احببت هذه السللعه كثيرا ممتازه 1
لم اتوقعها هكذا كنت اتمناها بمواصفات ذكية تناسب مستوى الايفون 0
الجهاز رائع جدا واصلي ومعه ضمان عامان 1
قمت بشراء أكثر من عطر من العربية للعود مثل كلماتي و سحر الكلمات و مستي وود ثبات الرائحة لهذه العطور غير جيدة إذا كنت تبحث عن عطر يدوم وقت أكثر على الجسم أو الملابس فعطور العربية للعود ليست اختيار مناسب على الاطلاق 0
سلعه جيده لكنها باااهظة الثمن وهي صناعه صينيه 0
جيدة جدا وجميلة وانصح الجميع بها وهي لاتسبب خشونه لشعر واوالوانها زاهية تماما ورائعة لمناسبات والاعياد 1
انها عملية وجيدة وسهلة التنظيف 1


In [ ]:
#save the dataset
with open('product_review.txt', 'w') as f:
  for i in range(len(reviews)):
    f.write(f"{reviews[i]}, {labels[i]} '\n'")

## Create Sequences
Create sequences by using the most repeated 500 words

In [ ]:
tknzr = Tokenizer(lower=True, split=" ")
tknzr.fit_on_texts(reviews)

#making sequences:
X = tknzr.texts_to_sequences(reviews)
X = pad_sequences(X, padding='post', value=0)

## Create Numpy Arrays

In [ ]:
X = np.array(X)
y = np.array(labels)

print(X.shape)

(1648, 113)


## Create the model

In [ ]:
model = Sequential()
model.add(Embedding(len(tknzr.word_index), 32))
model.add(Bidirectional(GRU(units = 32)))
model.add(Dense(32, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          217056    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12672     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 231,841
Trainable params: 231,841
Non-trainable params: 0
__________________________________________________

## Train the model

In [ ]:
model.fit(X, y, validation_split = 0.1, epochs = 7, batch_size= 128, shuffle = True)

Epoch 1/7
12/12 [==============================] - 9s 137ms/step - loss: 0.6918 - accuracy: 0.5145 - val_loss: 0.6890 - val_accuracy: 0.7030
Epoch 2/7
12/12 [==============================] - 1s 49ms/step - loss: 0.6791 - accuracy: 0.7269 - val_loss: 0.6804 - val_accuracy: 0.6424
Epoch 3/7
12/12 [==============================] - 1s 49ms/step - loss: 0.6405 - accuracy: 0.8564 - val_loss: 0.6342 - val_accuracy: 0.7697
Epoch 4/7
12/12 [==============================] - 1s 50ms/step - loss: 0.5193 - accuracy: 0.8982 - val_loss: 0.5324 - val_accuracy: 0.7939
Epoch 5/7
12/12 [==============================] - 1s 50ms/step - loss: 0.2961 - accuracy: 0.9252 - val_loss: 0.4795 - val_accuracy: 0.7939
Epoch 6/7
12/12 [==============================] - 1s 50ms/step - loss: 0.1659 - accuracy: 0.9514 - val_loss: 0.4697 - val_accuracy: 0.8121
Epoch 7/7
12/12 [==============================] - 1s 49ms/step - loss: 0.1128 - accuracy: 0.9676 - val_loss: 0.5130 - val_accuracy: 0.8182


## Tests

In [ ]:
class_names = ['سلبي' , 'إيجابي']
def classify(sentence):
  sentence = process_review(sentence)
  sequence = [tknzr.word_index[word] for word in sentence.split(' ')]
  sequence = pad_sequences([sequence], maxlen = X.shape[1], padding='post', value=0)
  #print(sequence.dtype)
  #print(sequence)
  pred = model.predict(sequence)[0][0]
  print(class_names[np.round(pred).astype('int')], pred)
  

In [ ]:
classify("جميل")

إيجابي 0.7856761


In [ ]:
classify("السلعة كانت جيدة")

إيجابي 0.76657635


In [ ]:
classify("سيء")

سلبي 0.13229607


In [ ]:
classify("لا بأس بها")

سلبي 0.13391067


In [ ]:
classify("تفاجأت بجودة المنتج")

سلبي 0.36554196


In [ ]:
import csv
def create_csv(file, dict):
    with open(file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key in dict.keys():
            writer.writerow([key,dict[key]])

In [ ]:
create_csv("word2index.csv", tknzr.word_index)

In [ ]:
model.save("keras.h5")